In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Post-Traumatic_Stress_Disorder"
cohort = "GSE77164"

# Input paths
in_trait_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder"
in_cohort_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder/GSE77164"

# Output paths
out_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE77164.csv"
out_gene_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE77164.csv"
out_clinical_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE77164.csv"
json_path = "../../output/preprocess/Post-Traumatic_Stress_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral blood transcriptome profiles in Nepali child soldiers and civilians."
!Series_summary	"Analysis of transcript abundance estimates as a function of child soldier status, PTSD symptoms, and psychological resilience."
!Series_overall_design	"Gene expression profiling was conducted on dried blood spot (DBS) samples collected from community dwelling adolescents and young adults in Nepal.  Approximatley half of the sample were former child soldiers in the Nepal People's War and the other half were demographically similiar civilian non-combatants.  In addition to basic demographic characteristics (age, sex, ethnic minority status, social caste status, education level), participants were also assessed on syptoms of post-traumatic stress (PTS, assessed by a culturally adapted version of The Child PTSD Symptom Scale; Kohrt BA, et al. (2011) Validation of cross-cultural child mental health and psychosocial research instruments: adapting the Depres

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset and extract clinical features

# 1. Gene Expression Data Availability
# Examining cd3d, cd3e, cd4, cd8a, etc. in the sample characteristics
# These are gene expression values, and this is likely gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# PTS (Post-Traumatic Stress) is our trait of interest (row 6)
trait_row = 6  # PTS (Post-Traumatic Stress)
age_row = 2    # Age data available in row 2
gender_row = 1  # Gender data is in row 1 (female: 0/1)

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert PTS (Post-Traumatic Stress) data to binary values."""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
        try:
            # The pts values are already 0 and 1
            return int(value)
        except (ValueError, TypeError):
            return None
    return None

def convert_age(value):
    """Convert age data to continuous values."""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
        try:
            return int(value)
        except (ValueError, TypeError):
            return None
    return None

def convert_gender(value):
    """Convert gender data to binary values (0=female, 1=male)."""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
        # Based on the data, 'female: 1' means female, 'female: 0' means male
        try:
            # Invert the value since 1 corresponds to female in the data
            # but we want 0=female, 1=male
            return 1 - int(value)
        except (ValueError, TypeError):
            return None
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create clinical data from the sample characteristics dictionary
    sample_characteristics = {
        0: ['childsoldier: 0', 'childsoldier: 1'],
        1: ['female: 1', 'female: 0'],
        2: ['age: 19', 'age: 22', 'age: 20', 'age: 18', 'age: 23', 'age: 21', 'age: 17', 'age: 16', 'age: 15', 'age: 24', 'age: 26'],
        3: ['ethnicminority: 0', 'ethnicminority: 1'],
        4: ['castelow: 1', 'castelow: 0'],
        5: ['educationlevel: 3', 'educationlevel: 2', 'educationlevel: 5', 'educationlevel: 4', 'educationlevel: 0', 'educationlevel: 6', 'educationlevel: 1'],
        6: ['pts: 0', 'pts: 1'],
        7: ['resilience: 6', 'resilience: 12', 'resilience: 11', 'resilience: 19', 'resilience: 7', 'resilience: 15', 'resilience: 14', 'resilience: 13', 'resilience: 9', 'resilience: 21', 'resilience: 8', 'resilience: 16', 'resilience: 10', 'resilience: 4', 'resilience: 17', 'resilience: 18', 'resilience: 20', 'resilience: 5', 'resilience: 22', 'resilience: 2', 'resilience: 24']
    }
    
    # Convert sample characteristics into a DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_characteristics, orient='index')
    
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    print("Preview of selected clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to the specified output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{0: [0.0, 19.0, 0.0], 1: [1.0, 22.0, 1.0], 2: [nan, 20.0, nan], 3: [nan, 18.0, nan], 4: [nan, 23.0, nan], 5: [nan, 21.0, nan], 6: [nan, 17.0, nan], 7: [nan, 16.0, nan], 8: [nan, 15.0, nan], 9: [nan, 24.0, nan], 10: [nan, 26.0, nan], 11: [nan, nan, nan], 12: [nan, nan, nan], 13: [nan, nan, nan], 14: [nan, nan, nan], 15: [nan, nan, nan], 16: [nan, nan, nan], 17: [nan, nan, nan], 18: [nan, nan, nan], 19: [nan, nan, nan], 20: [nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE77164.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')

Gene data dimensions: 34271 genes × 254 samples


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers shown, let's analyze if they need mapping:

# The identifiers include recognized human gene symbols like:
# - A1BG (Alpha-1-B Glycoprotein)
# - A2M (Alpha-2-Macroglobulin)
# - AAAS (Aladin WD Repeat Nucleoporin)
# - AACS (Acetoacetyl-CoA Synthetase)

# These appear to be standard HUGO Gene Nomenclature Committee (HGNC) symbols
# for human genes, not probe IDs that would need mapping.

# Some identifiers like 7A5, AAA1 are less conventional but may be alternative gene symbols
# or specific transcript variants. However, the majority appear to be standard gene symbols.

# Therefore:
requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Re-extract the gene expression data to ensure we have it properly defined
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)

# Since requires_gene_mapping = False from Step 4, we'll skip mapping and directly normalize gene symbols
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 normalized gene symbols: {normalized_gene_data.index[:5].tolist() if len(normalized_gene_data) > 0 else 'No genes after normalization'}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical data - using the same approach as previous step
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# From Step 2, we identified the correct rows
trait_row = 6  # PTS (Post-Traumatic Stress)
age_row = 2    # Age data available in row 2
gender_row = 1  # Gender data is in row 1 (female: 0/1)

def convert_trait(value):
    """Convert PTS (Post-Traumatic Stress) data to binary values."""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
        try:
            # The pts values are already 0 and 1
            return int(value)
        except (ValueError, TypeError):
            return None
    return None

def convert_age(value):
    """Convert age data to continuous values."""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
        try:
            return int(value)
        except (ValueError, TypeError):
            return None
    return None

def convert_gender(value):
    """Convert gender data to binary values (0=female, 1=male)."""
    if value is None:
        return None
    if ":" in value:
        value = value.split(":")[1].strip()
        # Based on the data, 'female: 1' means female, 'female: 0' means male
        try:
            # Invert the value since 1 corresponds to female in the data
            # but we want 0=female, 1=male
            return 1 - int(value)
        except (ValueError, TypeError):
            return None
    return None

# Extract clinical features with correct row indices and conversion functions
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Debug: Show preview of clinical data
print("Preview of clinical data:")
print(preview_df(selected_clinical_df))

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait is biased
print("\nChecking for bias in the trait variable:")
# The trait in this dataset is binary (PTSD vs No PTSD)
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation
is_trait_available = True  # We confirmed trait data is available
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset studies gene expression in child soldiers with assessment of PTSD symptoms and psychological resilience."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: (20593, 254)
First 5 normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1']


Normalized gene data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE77164.csv


Preview of clinical data:
{'GSM2045182': [0.0, 19.0, 0.0], 'GSM2045183': [0.0, 22.0, 1.0], 'GSM2045184': [0.0, 20.0, 1.0], 'GSM2045185': [0.0, 18.0, 1.0], 'GSM2045186': [0.0, 22.0, 1.0], 'GSM2045187': [0.0, 23.0, 0.0], 'GSM2045188': [1.0, 21.0, 1.0], 'GSM2045189': [0.0, 22.0, 0.0], 'GSM2045190': [0.0, 19.0, 0.0], 'GSM2045191': [0.0, 22.0, 1.0], 'GSM2045192': [0.0, 20.0, 1.0], 'GSM2045193': [0.0, 18.0, 1.0], 'GSM2045194': [0.0, 20.0, 1.0], 'GSM2045195': [0.0, 21.0, 0.0], 'GSM2045196': [0.0, 21.0, 1.0], 'GSM2045197': [0.0, 21.0, 1.0], 'GSM2045198': [0.0, 18.0, 1.0], 'GSM2045199': [0.0, 22.0, 1.0], 'GSM2045200': [0.0, 23.0, 1.0], 'GSM2045201': [1.0, 22.0, 0.0], 'GSM2045202': [0.0, 17.0, 0.0], 'GSM2045203': [0.0, 19.0, 1.0], 'GSM2045204': [1.0, 18.0, 0.0], 'GSM2045205': [0.0, 19.0, 0.0], 'GSM2045206': [0.0, 19.0, 0.0], 'GSM2045207': [0.0, 21.0, 1.0], 'GSM2045208': [0.0, 20.0, 1.0], 'GSM2045209': [0.0, 21.0, 1.0], 'GSM2045210': [1.0, 20.0, 0.0], 'GSM2045211': [0.0, 21.0, 1.0], 'GSM2045212':

Data shape after handling missing values: (254, 20596)

Checking for bias in the trait variable:
For the feature 'Post-Traumatic_Stress_Disorder', the least common label is '1.0' with 29 occurrences. This represents 11.42% of the dataset.
The distribution of the feature 'Post-Traumatic_Stress_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 19.0
  50% (Median): 20.0
  75%: 21.75
Min: 15.0
Max: 26.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 113 occurrences. This represents 44.49% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE77164.csv
